# Demo de implementación de un Algoritmo Genético (AG) para resolver el problema de encontrar los exponentes de números primos para aproximar a un número determinado
Utiliza la librería DEAP disponible en https://deap.readthedocs.io/en/master/

0) Preparar el ambiente:

In [ ]:
#@title
!pip install deap

# instala paquetes gráficos
!apt-get install -y graphviz-dev
!pip install pygraphviz

1) Importar librerías:

In [ ]:
#@title 
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

from deap import base
from deap import creator
from deap import tools

import matplotlib
import matplotlib.pyplot as plt
import networkx
from IPython.display import display
from PIL import Image

print("\nLibrerías importadas")

2) Establecer parámetros generales de la Corrida:

In [ ]:
#@title PARÁMETROS

#@markdown Parámetros de la Factorización:

# el número a aproximar
VALOR_A_APROXIMAR =  66#@param {type:"integer"}

# parámetros para el cromosoma
VALORES_PRIMOS = "2,3,5,7,11" #@param {type:"string"}
PRIMOS_CON_EXPONENTE = [int(v) for v in VALORES_PRIMOS.split(",")]

VALOR_MAXIMO_EXPONENTE = 3 #@param {type:"integer"}

#@markdown Parámetros del Algoritmo Genético:

# Cantidad de Ciclos de la Corrida
CANT_CICLOS =  100#@param {type:"integer"}

# Indica que finaliza corrida cuando se alcance una  Aptitud Mínima (opcional)
FINALIZA_CORRIDA_POR_APTITUD = True  #@param {type:"boolean"}

# Cantidad de Individuos en la Población
CANT_INDIVIDUOS_POBLACION = 65  #@param {type:"slider", min:1, max:100, step:1}

print("\nParámetros de la Corrida definidos")

3) Inicializar objetos de DEAP para utilizar:

In [ ]:
#@title FUNCIÓN DE APTITUD Y ESTRUCTURA DEL CROMOSOMA

#@markdown > Función de Aptitud:
#@markdown - indica que es para maximizar.
#@markdown - determina la aptitud como la diferencia entre 100% menos el error relativo entre el valor a aproximar y el del individuo  (el mejor debería ser 100%).

import math

## Inicializa objeto Toolbox auxiliar
toolbox = base.Toolbox()

creator.create("Fitness", base.Fitness, weights=(1.0,))

# definimos la función de aptitud a evaluar
def funcAptitud(individuo):

    # calcula el número que corresponde al individuo
    valorIndividuo = formatearIndividuo(individuo, devStr=False)
    # calcula el error relativo
    if VALOR_A_APROXIMAR != 0:
      errorRelativo = abs(VALOR_A_APROXIMAR - valorIndividuo) / VALOR_A_APROXIMAR
    else:
      errorRelativo = abs(VALOR_A_APROXIMAR - valorIndividuo) / 0.000001      
    # ajuste para el cálculo de aptitud para no asignar valores negativos 
    # (genera problema con métodos de selección)
    if errorRelativo >= 1:
      auxLog = math.log(errorRelativo)
      if auxLog != 0:
        aptitud = 1.0 / math.log(errorRelativo)
      else:
        aptitud = 0.0
    else:  
      aptitud = 100.0 - (errorRelativo*100.0)
    return aptitud,


# función para mostrar aptitud
def formatearAptitud(fitness):
    return round(fitness, 3)

# función para mostrar individuo
def formatearIndividuo(individuo, devStr=True, inclTotal=True):
    
    # calcula el número que corresponde al individuo
    strIndividuo = ""
    valorIndividuo = 1
    for i in range(len(individuo)):
      p = PRIMOS_CON_EXPONENTE[i]
      valorIndividuo = valorIndividuo * ( p ** individuo[i] )
      if (individuo[i] > 0):
        if len(strIndividuo)>0:
          strIndividuo = strIndividuo + " x "
        strIndividuo = strIndividuo + str(p) + "^" + str(individuo[i])
    # determina el resultado
    if devStr:
      if strIndividuo == "":
        res = "1"
      else:
        res = strIndividuo
      if inclTotal:
        res = res + " = " + str(valorIndividuo)    
    else:
      res = valorIndividuo
    return res

# registra la función que se va a evaluar
toolbox.register("evaluate", funcAptitud)

print("\nFunción de Aptitud definida")

#@markdown > Estructura del Cromosoma:
#@markdown - indica que es una lista de valores enteros (para simplificar la representación del problema).
# indica que los individuos son una lista de genes que aplica la función antes definida
creator.create("Individual", list, fitness=creator.Fitness)

# indica que los genes son int correspondiente a las exponentes a encontrar
toolbox.register("attr_int", random.randint, 0, VALOR_MAXIMO_EXPONENTE) 

# registra el tipo de individuo y población a usar
cant_genesCromosoma = len(PRIMOS_CON_EXPONENTE)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, cant_genesCromosoma)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Determina mínima Heuristica para finalizar el algoritmo
MIN_HEURISTICA_FINALIZAR = 100.0

print("\nCromosoma definido")

In [ ]:
#@title OPERADORES GENÉTICOS
#@markdown definidos  en https://deap.readthedocs.io/en/master/api/tools.html#operators 

tipo_seleccion = 'Torneo' #@param ["Azar", "Torneo", "Ranking", "Ruleta"]
tipo_cruzamiento = 'Azar' #@param ["Simple-1Punto", "Simple-2Puntos", "Azar"]

# Probabilidad del Mutación 
probab_mutacion = 0.5 #@param {type:"slider", min:0, max:1, step:0.05}

# Registrar tipo de Selección a aplicar con sus parámetros
if tipo_seleccion == 'Torneo':
  toolbox.register("select", tools.selTournament, tournsize=2)
elif tipo_seleccion == 'Ranking':
  toolbox.register("select", tools.selBest)
elif tipo_seleccion == 'Ruleta':
  toolbox.register("select", tools.selRoulette)
else:
  toolbox.register("select", tools.selRandom)

# Registrar tipo de Cruzamiento a aplicar con sus parámetros
if tipo_cruzamiento == 'Simple-1Punto':
  toolbox.register("mate", tools.cxOnePoint)
elif tipo_cruzamiento == 'Simple-2Puntos':
  toolbox.register("mate",  tools.cxTwoPoint)  
else:
  toolbox.register("mate", tools.cxUniform, indpb=1/cant_genesCromosoma)

# Registrar tipo de Mutación a aplicar con sus parámetros
toolbox.register("mutate", tools.mutUniformInt, low=0, up=VALOR_MAXIMO_EXPONENTE, indpb=10/cant_genesCromosoma)

print("\nOperadores Genéticos definidos")

4) Ejecutar la corrida del algoritmo mostrando resultados:

In [ ]:
#@title FUNCIONES AUXILIARES

## Define una función auxiliar para calcular estadísticas y guarda info en vectores auxiliares
def CalculoEstadisticas(ciclo, indivPobla, muestra, mejorMax = True):   
    global mejorIndCorrida, mejorIndCorrida_ciclo
    global ciclosMaxAptitud
    global ciclosPromAptitud
    global ciclosMinAptitud
    if len(indivPobla) == 0:
      return None, 0, 0, 0 
    auxMax = None
    auxMin = None
    auxSum = 0
    auxBestInd = None
    auxBestIndApt = None    
    for ind in indivPobla:
        apt = round(ind.fitness.values[0], 2)
        auxSum = auxSum + apt
        if (auxMax == None) or (apt > auxMax):
            auxMax = apt
            if mejorMax:
              auxBestInd = ind
              auxBestIndApt = apt
        if (auxMin == None) or (apt < auxMin):
            auxMin = apt
            if not mejorMax:
              auxBestInd = ind
              auxBestIndApt = apt
    auxProm = round(auxSum / len(indivPobla), 2)
    if (mejorIndCorrida == None) or (mejorIndCorrida.fitness.values < auxBestInd.fitness.values):
          mejorIndCorrida = auxBestInd
          mejorIndCorrida_ciclo = ciclo
    ciclosMaxAptitud.append( auxMax )
    ciclosPromAptitud.append( auxProm )
    ciclosMinAptitud.append( auxMin )
    if muestra:          
        #print("\n-- Ciclo  %i --" % ciclo)
        print(" ++ Mejor Individuo:", formatearIndividuo(auxBestInd), " {", formatearAptitud(auxBestIndApt), "}")
        print("      Max: ", formatearAptitud(auxMax), " / Promedio: ", formatearAptitud(auxProm), " / Min: ", formatearAptitud(auxMin))
    return auxBestInd, auxMax, auxProm, auxMin


def asignarAptitud(indivPobla):
  # Asigna el valor de aptitud a los individuos 
  fitnesses = list(map(toolbox.evaluate, indivPobla))
  for ind, fit in zip(indivPobla, fitnesses):
      ind.fitness.values = fit


def mostrarParamsDetalle():
  print("EJECUCIÓN DE LA CORRIDA: ")
  print("\t Criterio de Paro: "+str(CANT_CICLOS)+" ciclos"+(", o si llega a la solución" if FINALIZA_CORRIDA_POR_APTITUD else "")+".")
  print("\t Tamaño de la Población: "+str(CANT_INDIVIDUOS_POBLACION)+" individuos.")
  print("\t Método de Selección: "+tipo_seleccion+".")
  print("\t Método de Cruzamiento: "+tipo_cruzamiento+".")
  print("\t Método de Mutación: Simple con "+str(probab_mutacion*100)+"% de probabilidad.")
  print("")  


def mostrarPoblaDetalle(indivPobla, titulo=""):
  if titulo != "":
    print("\n" + titulo)
  for ind in indivPobla:
    print("\t", formatearIndividuo(ind), " { ", formatearAptitud(ind.fitness.values[0]), " } ")
  print("")  

print("\nFunciones auxiliares definidas")

In [ ]:
#@title EJECUCIÓN DE LA CORRIDA

# Indica si se muestra el detalle de los operadores genéticos aplicados
MOSTRAR_DETALLE = True #@param {type:"boolean"}

# Indica si se muestra el detalle de las estadísticas en cada ciclo
MUESTRA_ESTADISTICAS = True  #@param {type:"boolean"}

if MOSTRAR_DETALLE:
  mostrarParamsDetalle()

# Resetear Genelogy
history = tools.History()
# Decorate the variation operators
toolbox.decorate("mate", history.decorator)
toolbox.decorate("mutate", history.decorator)

# cálcula cantidad de individuos seleccionados como la mitad + 1
cant_ind_selecc = (CANT_INDIVIDUOS_POBLACION//2)+1

# Define la población inicial
indivPobla = toolbox.population(n=CANT_INDIVIDUOS_POBLACION)
history.update(indivPobla)

# Asigna el valor de aptitud a los individuos de la población inicial
asignarAptitud(indivPobla)

if MOSTRAR_DETALLE or MUESTRA_ESTADISTICAS:
  print("\n-- Ciclo  0 --")

if MOSTRAR_DETALLE:
  mostrarPoblaDetalle(indivPobla, "*) Población Inicial:")

# vectores auxiliares 
ciclo = 1    
mejorIndCorrida = None
mejorIndCorrida_ciclo = 0
ciclosIndiv = []
ciclosMaxAptitud = []
ciclosPromAptitud = []
ciclosMinAptitud = []

# Cálcula estadísticas y guarda info en vectores auxiliares
auxBestInd, auxMax, auxProm, auxMin = CalculoEstadisticas(0, indivPobla, MUESTRA_ESTADISTICAS)
      
        # criterio de paro
while (ciclo < CANT_CICLOS) \
  and (not(FINALIZA_CORRIDA_POR_APTITUD) or (mejorIndCorrida==None) or (mejorIndCorrida.fitness.values[0]<MIN_HEURISTICA_FINALIZAR)):

    if MOSTRAR_DETALLE or MUESTRA_ESTADISTICAS:
      print("\n-- Ciclo  %i --" % ciclo)

    # Realiza la Selección
    indivSelecc = toolbox.select(indivPobla, cant_ind_selecc)
    if MOSTRAR_DETALLE:
      mostrarPoblaDetalle(indivSelecc, "a) Individuos Seleccionados:")
    
    # Realiza el Cruzamiento
    # por cada individuo seleccionado los clona (sino pueden afectarle los cambios)
    # y luego le busca una pareja al azar dentro de los seleccionados para generar 2 hijos
    if MOSTRAR_DETALLE:
      print("b) Cruzamiento: ")
    indivHijos = []
    for hijo1 in list(map(toolbox.clone, indivSelecc)):
      if len(indivHijos) < len(indivPobla):
        # todavía falta generar hijos
        hijo2 = list(map(toolbox.clone, random.sample(indivSelecc, 1)))[0]
        if MOSTRAR_DETALLE:
          print("\t", formatearIndividuo(hijo1, inclTotal=False), "*", formatearIndividuo(hijo2, inclTotal=False), end=" =")
        # hace el cruzamiento
        toolbox.mate(hijo1, hijo2)
        del hijo1.fitness.values
        del hijo2.fitness.values        
        # agrega 1er hijo
        indivHijos.append(hijo1)
        if MOSTRAR_DETALLE:
          print("", formatearIndividuo(hijo1, inclTotal=False), end="")
        if len(indivHijos) < len(indivPobla):
          # agrega 2do hijo
          indivHijos.append(hijo2)
          if MOSTRAR_DETALLE:
            print(" &", formatearIndividuo(hijo2, inclTotal=False), end="")
        if MOSTRAR_DETALLE:
          print("")
      else:
        break

    # controla que mantenga que los hijos 
    # tenga la misma cantidad de individuos que la población original
    if len(indivHijos) != len(indivPobla):
      print("ERROR: distinta cantidad de individuos hijos ", len(indivHijos) , " | ", len(indivPobla))

    # Realiza la Mutación
    if MOSTRAR_DETALLE:
      print("\nc) Mutación: ")
    if random.random() < probab_mutacion:
      # ejecuta mutación:
      # selecciona al azar el individuo a mutar
      # y lo muta aplicando el método      
      mutant = indivHijos[random.randint(0, len(indivHijos)-1)]
      if MOSTRAR_DETALLE:
        print("\tse ejecuta: ", formatearIndividuo(mutant, inclTotal=False), end=" ")
      # hace la mutación
      toolbox.mutate(mutant)
      del mutant.fitness.values
      if MOSTRAR_DETALLE:
        print("--> ", formatearIndividuo(mutant, inclTotal=False), "\n")
    else:
      # no se ejecuta mutación
      print("\tNo se ejecuta\n")

    # Evalua a los individuos hijos que salen de Cruzamiento & Mutación
    #  para determinar si son válidos y su valor de aptitud
    invalid_ind = [ind for ind in indivHijos if not ind.fitness.valid]
    asignarAptitud(invalid_ind)

    # Reemplaza la población actual con los hijos
    indivPobla[:] = indivHijos    
    if MOSTRAR_DETALLE:
      mostrarPoblaDetalle(indivPobla, "*) Siguiente Generación:")

    # Cálcula estadísticas y guarda info en vectores auxiliares
    auxBestInd, auxMax, auxProm, auxMin = CalculoEstadisticas(ciclo, indivPobla, MUESTRA_ESTADISTICAS)

    ciclo = ciclo + 1

print("\n-- Corrida Finalizada en %i ciclos --" % ciclo )
print("== Mejor Individuo de la Corrida aparece en ciclo ", mejorIndCorrida_ciclo, ":", formatearIndividuo(mejorIndCorrida), " { ", formatearAptitud(mejorIndCorrida.fitness.values[0]), " }  ==")

5) Generar gráficos de la corrida:

In [ ]:
#@title MOSTRAR GRÁFICO DE LA CORRIDA
plt.figure(figsize=(15,8)) 
plt.plot(ciclosPromAptitud)
plt.plot(ciclosMinAptitud)
plt.plot(ciclosMaxAptitud)
plt.title('Resultados de la Corrida')
plt.xlabel('Ciclos')
plt.ylabel('Aptitud')
plt.legend(['Promedio', 'Mínima', 'Máxima'], loc='lower right')
plt.grid(True)
USAR_ESCALA_LOG = True #@param {type:"boolean"}
if USAR_ESCALA_LOG:
  plt.yscale('log')
plt.show()


In [ ]:
#@title MOSTRAR ÁRBOL GENEALÓGICO

#@markdown Nota: el nodo indica el Valor Aproximado calculado por los valores del cromosoma.

incluir_Id_Individuo = False #@param {type:"boolean"}

pintar_Color_Aptitud = True #@param {type:"boolean"}
#@markdown (si se activa: color más ROJO significa PEOR aptitud, y más VERDE es MEJOR aptitud)

# genera gráfico 
graph = networkx.MultiDiGraph(history.genealogy_tree)
graph = graph.reverse()     # Make the graph top-down

# cambia ID de individuos por detalle cromosoma 
indHistory = {}
indApt = {}
for k in history.genealogy_history.keys():
  auxStrLabel = str(formatearIndividuo(history.genealogy_history[k], inclTotal=False))
  if incluir_Id_Individuo:
    auxStrLabel = "(" + str(k) + ") " + auxStrLabel
  indHistory[k] = auxStrLabel
  if pintar_Color_Aptitud:
    indApt[auxStrLabel] = toolbox.evaluate(history.genealogy_history[k])[0]
graph = networkx.relabel_nodes(graph, indHistory)

# muestra como árbol
A = networkx.nx_agraph.to_agraph(graph)

# cambia colores por valor de aptitud
if pintar_Color_Aptitud:
  # determina mapa colores y valores mínimos/máximos
  cmap = matplotlib.cm.get_cmap('RdYlGn')  
  ## para usar cuando no se conoce mín y/o max
  ##kMax= max(indApt.keys(), key=(lambda new_k: indApt[new_k])) 
  ##kMin = min(indApt.keys(), key=(lambda new_k: indApt[new_k])) 
  ##norm = matplotlib.colors.Normalize(vmin=indApt[kMin], vmax=indApt[kMax])
  norm = matplotlib.colors.Normalize(vmin=0.00, vmax=100.0)
  # indica cambia color del nodo
  A.node_attr['style']='filled'
  for k in A.nodes():
      # obtiene el nodo
      n =  A.get_node(k)
      if k in indApt:
        # a partir de la aptitud asigna el color
        color = cmap(norm(indApt[k]))
        # asigna en format HEX
        n.attr['fillcolor'] =  matplotlib.colors.rgb2hex(color)

# define layout del gráfico
A.layout('dot', args='-Nfontsize=10 -Nwidth="1.0" -Nheight=".4" -Nmargin=0 -Gfontsize=8')

# genera el gráfico
auxFile = 'genHistory.png'
A.draw(auxFile)

# se agrega para evitar error por superar cantidad máxima de pixeles
import PIL.Image
PIL.Image.MAX_IMAGE_PIXELS = 933120000

# muestra el gráfico
display(Image.open(auxFile))
